In [1]:
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as SPLIT
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('combined_cleaned.csv')



In [3]:
# modify birthyear-> 0-1 on a float scale
birthyears = df.birthyear.unique()
_min = min(birthyears)
_max = max(birthyears)

def normalize_birthyear(year):
    return (year-_min)/(_max-_min)

df.birthyear = df.birthyear.apply(normalize_birthyear)
df.head()

,Unnamed: 0,text,birthyear,fame,gender,occupation
0,0,back looking circa early coeur dalene idaho bu...,0.647059,superstar,female,performer
1,1,last presidential election turned fewer 100 00...,0.308824,superstar,male,politics
2,2,angels amazing thanks see tonight seattle tayl...,0.720588,superstar,female,performer
3,3,listen remember us #todaystophits president do...,0.676471,superstar,female,creator
4,4,happy island vote madeira sure best island wor...,0.661765,superstar,male,manager


In [4]:
labels_to_onehot = ['fame', 'gender', 'occupation']

from sklearn.preprocessing import LabelBinarizer

labels = {}

for label in labels_to_onehot:
    unique_classes = len(df[label].unique())
    print('{} unique classes in {}'.format(unique_classes, label))
    labels_nd = df[label].values
    encoder = LabelBinarizer()
    labels[label] = encoder.fit_transform(labels_nd)


2 unique classes in fame
3 unique classes in gender
8 unique classes in occupation


In [5]:
texts = df.text
fame_labels = labels['fame']
gender_labels = labels['gender']
occ_labels = labels['occupation']
print(fame_labels)
print(gender_labels)
print(occ_labels)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]
[[1 0 0]
 [0 1 0]
 [1 0 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [8]:
from keras.preprocessing.text import Tokenizer
vocab_size = 15000
 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df.text)
 
x_train = tokenizer.texts_to_matrix(df.text, mode='tfidf')


In [12]:
fame_labels = to_categorical(fame_labels)
gender_labels = to_categorical(gender_labels)
occ_labels = to_categorical(occ_labels)

print(fame_labels.shape)
print(gender_labels.shape)
print(occ_labels.shape)

(6996, 2, 2)
(6996, 3, 2, 2)
(6996, 8, 2, 2)


In [10]:
#model = Sequential()
from keras.layers import Input, Dense, Dropout
from keras.models import Model

shape = (vocab_size,)
input_layer = Input(shape)
# before splitting categories
branch_layer = Dense(units=128, activation='relu')(input_layer)

# birthyear
_ = Dense(units=128, activation='relu')(branch_layer)
birthyear_out = Dense(units=1, activation='sigmoid', name='birthyear_out')(_)

# fame
_ = Dense(units=128, activation='relu')(branch_layer)
fame_out = Dense(units=len(fame_labels[0]), activation='sigmoid', name='fame_out')(_)

# gender
_ = Dense(units=128, activation='relu')(branch_layer)
gender_out = Dense(units=len(gender_labels[0]), activation='softmax', name='gender_out')(_)

# occupation
_ = Dense(units=128, activation='relu')(branch_layer)
occ_out = Dense(units=len(occ_labels[0]), activation='softmax', name='occ_out')(_)

model = Model(input=input_layer, outputs=[birthyear_out, fame_out, gender_out, occ_out])
#model = Model(input=input_layer, outputs=fame_out)
loss_fn = 'categorical_crossentropy'

model.compile(optimizer='adam',
              loss={'birthyear_out': 'mse', 'fame_out': loss_fn, 'gender_out': loss_fn, 'occ_out': loss_fn},
              metrics={'birthyear_out': 'mae', 'fame_out': 'accuracy', 'gender_out': 'accuracy', 'occ_out': 'accuracy'}
             )

#model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\Tollef\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15000)        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          1920128     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

In [11]:
model.fit(x_train, [df.birthyear.values, fame_labels, gender_labels, occ_labels], epochs=2, batch_size=32)




ValueError: Error when checking target: expected gender_out to have 2 dimensions, but got array with shape (6996, 3, 2)

In [ ]:
model.fit(x_train, _fame, epochs=2, batch_size=32)

In [ ]:
df.birthyear.values

In [ ]:
gender_labels.shape